In [102]:
import iris
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import iris.quickplot as qplt
import iris.plot as iplt
import itertools
import datetime
import glob
import os, sys
import cartopy.crs as ccrs
%matplotlib

Using matplotlib backend: Qt5Agg


In [103]:
forecast_date_time = datetime.datetime(2021,9, 20, 12)
str_year, str_month, str_day, str_hour = str(forecast_date_time.year), \
                                             str('%02d' % forecast_date_time.month), \
                                             str('%02d' % forecast_date_time.day), \
                                             str('%02d' % forecast_date_time.hour)

date_label = '%s%s%s_%sZ' % (str_year, str_month, str_day, str_hour)

In [104]:
member='000'
#out_data_dir = os.path.join(data_paths.dirs('mog_forecast_out_dir'), str_year, str_month, str_day)
out_data_dir = os.path.join('/scratch/hadpx/cold_surge_monitoring/mogreps/raw_data/', str_year, str_month, str_day, str_hour, member)

In [105]:
fc_times_all=np.arange(0, 192, 24)

In [106]:
iloc=0
filename = os.path.join(out_data_dir, 'englaa_profiles_pd%s.pp' %str('%03d' % fc_times_all[iloc]))
print(filename)
if os.path.exists(filename):
    cube = iris.load_cube(filename, 'relative_humidity')
    cube = cube.intersection(latitude=(-30,30), longitude=(80, 140))

/scratch/hadpx/cold_surge_monitoring/mogreps/raw_data/2021/09/20/12/000/englaa_profiles_pd000.pp


In [107]:
cube
#speed_cube = (u850_cube**2 + v850_cube**2)**0.5

<iris 'Cube' of relative_humidity / (%) (pressure: 9; latitude: 320; longitude: 214)>

In [108]:
start_coords = (120, 20)
end_coords = (100, -10)

In [109]:
def find_nearest_in_list(myList, myNumber):
    return min(myList, key=lambda x:abs(x-myNumber))

In [110]:
def get_cube_coords_along_line(cube, start_coords=None, end_coords=None):
    '''
    Generate a (lat, lon) pair of cube coordinates along a line
    made up of start and end points
    '''
    if not (start_coords is None) or (end_coords is None):
        lons = cube.coord('longitude').points
        lats = cube.coord('latitude').points
        
        # Find the actual coordinate values between the points
        # Bit of overkill, but want to make sure there are no round off errors
        # when calling the lat/lon values in the extraction step
        #r_lons = [find_nearest_in_list(lons, rlon) for rlon in np.arange(find_nearest_in_list(lons, start_coords[0]), 
        #                                                      find_nearest_in_list(lons, end_coords[0]), dlon)]
        #r_lats = [find_nearest_in_list(lats, rlat) for rlat in np.arange(find_nearest_in_list(lats, start_coords[1]), 
        #                                                      find_nearest_in_list(lats, end_coords[1]), dlat)]
        
        # Find the actual coordinate values between the points
        lon1, lat1 = start_coords
        lon2, lat2 = end_coords
        if lon1 < lon2:
            r_lons = lons[(lon1<lons) & (lons<=lon2)]
        else:
            r_lons = lons[(lon2<lons) & (lons<=lon1)][::-1]

        if lat1 < lat2:
            r_lats = lats[(lat1<lats) & (lats<=lat2)]
        else:
            r_lats = lats[(lat2<lats) & (lats<=lat1)][::-1]
            
        #print(r_lats)
        # Generate a line from start to end points
        #slope of the line y = mx + b
        #m = slope = (y1-y2)/(x1-x2)
        m = (r_lats[-1] - r_lats[0])/(r_lons[-1]-r_lons[0])

        #b = y-intercept = (x1*y2 - x2*y1)/(x1-x2)
        b = (r_lons[0]*r_lats[-1] - r_lons[-1]*r_lats[0])/(r_lons[0] - r_lons[-1])
        
        # lat points for all lon points along the line
        ys = [m*x+b for x in r_lons]
        
        # get the corresponding lat values from the coordinate list
        ys_lats = [find_nearest_in_list(lats, yy) for yy in ys]
        
        return r_lons, ys_lats
    else:
        print('Start/End coordinates missing.')
    

In [111]:
def cross_section_along_path(cube, lons=None, lats=None):
    cross_section = []
    # Extracting each point and appending
    for ylat, xlon in zip(lats, lons):
        cross_section.append(cube.extract(iris.Constraint(latitude=ylat, longitude=xlon)))
        
    # merging to a single cube
    # longitude values is given as aux coordinate
    return iris.cube.CubeList(cross_section).merge_cube()

In [112]:
xlons, ylats = get_cube_coords_along_line(cube, start_coords=start_coords, end_coords=end_coords)

In [113]:
cross_sect_cube = cross_section_along_path(cube, lons=xlons, lats=ylats)

In [114]:
if ('latitude' in [cd.standard_name for cd in cross_sect_cube.dim_coords]) and ('longitude' in [cd.standard_name for cd in cross_sect_cube.aux_coords]):
    section_dim_coord = cross_sect_cube.coord('latitude')
    section_aux_coord = cross_sect_cube.coord('longitude')
elif ('latitude' in [cd.standard_name for cd in cross_sect_cube.aux_coords]) and ('longitude' in [cd.standard_name for cd in cross_sect_cube.dim_coords]):
    section_dim_coord = cross_sect_cube.coord('longitude')
    section_aux_coord = cross_sect_cube.coord('latitude')
else:
    print('Something wrong with coordinates.')     

In [115]:
pressure_coord_points = cross_sect_cube.coord('pressure').points
dim_coord_points = section_dim_coord.points
aux_coord_points = section_aux_coord.points

P, L = np.meshgrid(pressure_coord_points, dim_coord_points)


In [116]:
dim_coord_points

array([-9.84375, -9.46875, -8.90625, -8.53125, -8.15625, -7.78125,
       -7.21875, -6.84375, -6.46875, -6.09375, -5.53125, -5.15625,
       -4.78125, -4.21875, -3.84375, -3.46875, -3.09375, -2.53125,
       -2.15625, -1.78125, -1.40625, -0.84375, -0.46875, -0.09375,
        0.46875,  0.84375,  1.21875,  1.59375,  2.15625,  2.53125,
        2.90625,  3.28125,  3.84375,  4.21875,  4.59375,  4.96875,
        5.53125,  5.90625,  6.28125,  6.84375,  7.21875,  7.59375,
        7.96875,  8.53125,  8.90625,  9.28125,  9.65625, 10.21875,
       10.59375, 10.96875, 11.53125, 11.90625, 12.28125, 12.65625,
       13.21875, 13.59375, 13.96875, 14.34375, 14.90625, 15.28125,
       15.65625, 16.21875, 16.59375, 16.96875, 17.34375, 17.90625,
       18.28125, 18.65625, 19.03125, 19.59375, 19.96875])

In [117]:
# definitions for the axes
left, width = 0.05, 0.4
bottom, height = 0.1, 0.7
spacing = 0.065

rect_map = [left, bottom, width, height]
rect_profile = [left, bottom + height + spacing, width, 0.4]

fig = plt.figure(figsize=(12, 6), dpi=100)

ax1 = fig.add_axes(rect_map, projection=ccrs.PlateCarree())
pos1 = ax1.get_position() # get the original position 
clevels = np.arange(0, 110, 10)
cf = iplt.contourf(cube.extract(iris.Constraint(pressure=700)), cmap='RdBu', levels=clevels)
ax1.scatter(xlons, ylats, color='orange', alpha=0.7)
ax1.plot([start_coords[0], end_coords[0]], [start_coords[1], end_coords[1]], color='red', alpha=0.7)
ax1.set_title('HHHH')
#ax1.colorbar(cf, orientation='horizontal')

ax1 = plt.gca()
ax1.coastlines('50m', alpha=0.5)
ax1.stock_img()
ax1.set_ylim([-10, 25])
ax1.set_xlim([90, 130])
gl = ax1.gridlines(draw_labels=True, alpha=0.5)
gl.xlabels_top = False
gl.ylabels_right = False


pos2 = [pos1.x0 + pos1.width+ spacing, pos1.y0,  pos1.width+spacing, pos1.height]
ax2 = fig.add_axes(pos2)

CS = plt.contourf(L, P, cross_sect_cube.data, cmap='RdBu', levels=clevels)
plt.colorbar(CS, orientation='horizontal')

xlocs = ax2.get_xticks()
ylocs = np.interp(xlocs, dim_coord_points, aux_coord_points)

x_labels = ['%.0f\n%.0f' %(np.floor(xx), np.floor(yy)) for xx, yy in zip(xlocs, ylocs)]

#ax2.set_xtick(dim_coord_points[::ticks_interval], x_labels)
ax2.set_xticks(xlocs)
ax2.set_xticklabels(x_labels)
ax2.tick_params(axis="x", labelsize=8) 
ax2.tick_params(axis="y", labelsize=8) 
ax2.invert_yaxis()
ax2.grid(alpha=0.5)
#plt.tight_layout()


In [118]:
np.interp(xlocs, dim_coord_points, aux_coord_points)
xlocs, 

(array([-10.,  -5.,   0.,   5.,  10.,  15.,  20.]),)

In [119]:
'''
plt.subplot(122)
cf1 = iplt.contourf(cross_sect_cube, cmap='RdBu')
plt.colorbar(cf1, orientation='horizontal')
plt.gca().invert_yaxis()
xlocs, labels = plt.xticks()
# Create names on the x axis
plt.xticks(xlocs, x_labels)
plt.xlim([min(x_vals_from_coord), max(x_vals_from_coord)])
'''

"\nplt.subplot(122)\ncf1 = iplt.contourf(cross_sect_cube, cmap='RdBu')\nplt.colorbar(cf1, orientation='horizontal')\nplt.gca().invert_yaxis()\nxlocs, labels = plt.xticks()\n# Create names on the x axis\nplt.xticks(xlocs, x_labels)\nplt.xlim([min(x_vals_from_coord), max(x_vals_from_coord)])\n"

In [120]:
[label for label in labels

SyntaxError: unexpected EOF while parsing (<ipython-input-120-b62e6b640c5d>, line 1)

In [594]:
def return_indices_of_a(a, b):
    '''
    Function to return indices of elements in A that matches values in B
    :param a: list a
    :type a:
    :param b: list b
    :type b:
    :return: indices of elements in a that matches values in b
    :rtype:
    '''
    b_set = set(b)
    return [i for i, v in enumerate(a) if v in b_set]

In [595]:
nprofiles, nlevels = cross_sect_cube.shape

if ('latitude' in [cd.standard_name for cd in cross_sect_cube.dim_coords]) and ('longitude' in [cd.standard_name for cd in cross_sect_cube.aux_coords]):
    section_dim_coord = cross_sect_cube.coord('latitude')
    section_aux_coord = cross_sect_cube.coord('longitude')
elif ('latitude' in [cd.standard_name for cd in cross_sect_cube.aux_coords]) and ('longitude' in [cd.standard_name for cd in cross_sect_cube.dim_coords]):
    section_dim_coord = cross_sect_cube.coord('longitude')
    section_aux_coord = cross_sect_cube.coord('latitude')
else:
    print('Something wrong with coordinates.')                                                                              

In [596]:
x_vals_from_coord = [find_nearest_in_list(section_dim_coord.points, xl) for xl in xlocs]
inds = return_indices_of_a([pt for pt in section_dim_coord.points], x_vals_from_coord)
#section_dim_coord.points
xlocs, section_dim_coord.points[inds], section_aux_coord.points[inds]

x_labels = ['%.1f\n%.1f' %(section_dim_coord.points[ind], section_aux_coord.points[ind]) for ind in inds]
#x_labels = ['%.1f\n%.1f' %(xlocs[i], section_aux_coord.points[ind]) for i, ind in enumerate(inds)]
x_labels

['0.1\n100.3',
 '2.3\n102.5',
 '5.0\n105.0',
 '7.4\n107.6',
 '10.0\n110.1',
 '12.7\n112.6',
 '14.9\n114.9',
 '17.3\n117.4',
 '20.0\n120.0']

In [573]:
n_xlocs = len(xlocs)
n_xlocs
len(x_labels)

9

In [510]:
x_labels = ['%s\n%s' %(np.floor(cross_sect_cube.coord('longitude').points[n]), 
            np.floor(cross_sect_cube.coord('latitude').points[n])) for n in idx]
len(xlocs), len(x_labels)

(9, 9)

In [355]:
start_coords = (120, 25)
end_coords = (130, 0)

In [356]:
lons
lon1, lat1 = start_coords
lon2, lat2 = end_coords
if lon1 < lon2:
    print(lons[(lon1<lons) & (lons<=lon2)])
else:
    print(lons[(lon2<lons) & (lons<=lon1)][::-1])

if lat1 < lat2:
    print(lats[(lat1<lats) & (lats<=lat2)])
else:
    print(lats[(lat2<lats) & (lats<=lat1)][::-1])

[120.234375 120.515625 120.796875 121.078125 121.359375 121.640625
 121.921875 122.203125 122.484375 122.765625 123.046875 123.328125
 123.609375 123.890625 124.171875 124.453125 124.734375 125.015625
 125.296875 125.578125 125.859375 126.140625 126.421875 126.703125
 126.984375 127.265625 127.546875 127.828125 128.109375 128.390625
 128.671875 128.953125 129.234375 129.515625 129.796875]
[24.84375 24.65625 24.46875 24.28125 24.09375 23.90625 23.71875 23.53125
 23.34375 23.15625 22.96875 22.78125 22.59375 22.40625 22.21875 22.03125
 21.84375 21.65625 21.46875 21.28125 21.09375 20.90625 20.71875 20.53125
 20.34375 20.15625 19.96875 19.78125 19.59375 19.40625 19.21875 19.03125
 18.84375 18.65625 18.46875 18.28125 18.09375 17.90625 17.71875 17.53125
 17.34375 17.15625 16.96875 16.78125 16.59375 16.40625 16.21875 16.03125
 15.84375 15.65625 15.46875 15.28125 15.09375 14.90625 14.71875 14.53125
 14.34375 14.15625 13.96875 13.78125 13.59375 13.40625 13.21875 13.03125
 12.84375 12.65625 12.46

In [344]:
lon2, lat2

(120, 0)

In [192]:
qplt.contourf(cross_section_cube)
plt.gca().invert_yaxis()

In [211]:
cross_section_cube.coord('latitude').points, cross_section_cube.coord('longitude').points

(array([14.90625, 15.28125, 15.46875, 15.84375, 16.03125, 16.40625,
        16.59375, 16.96875, 17.15625, 17.53125, 17.71875, 18.09375,
        18.28125, 18.65625, 18.84375, 19.21875, 19.40625, 19.78125]),
 array([125.015625, 125.296875, 125.578125, 125.859375, 126.140625,
        126.421875, 126.703125, 126.984375, 127.265625, 127.546875,
        127.828125, 128.109375, 128.390625, 128.671875, 128.953125,
        129.234375, 129.515625, 129.796875]))

In [13]:
xlon = u850_cube.coord('longitude').points
ylat = u850_cube.coord('latitude').points
X, Y = np.meshgrid(xlon, ylat)

In [14]:
X.shape, u850_cube.shape, v850_cube.shape

((214, 178), (8, 36, 214, 178), (8, 36, 214, 178))

In [9]:
t = 2 # for t in times:

valid_date = forecast_date_time + datetime.timedelta(days=t)
valid_date_label = '%s/%s/%s' % (valid_date.year, str('%02d' % valid_date.month), str('%02d' % valid_date.day))
        
lw = 3 * speed_ens_mean[t].data / 20.0 #speed.data.max()

fig = plt.figure(figsize=(10, 10), dpi=100)

# contour plot of precip
clevels = [1, 5, 10, 15, 20, 25, 30, 35, 40]
cf = iplt.contourf(precip_ens_mean[t], levels=clevels, cmap='gray_r', alpha=0.65, extend='max')
axc = plt.gca()
axc.coastlines('50m', alpha=0.5)
axc.set_ylim([-10, 20])
axc.set_xlim([90, 130])
axc.stock_img()
axc.set_yticks([-10, 0, 10, 20], crs=ccrs.PlateCarree())
axc.set_xticks([90, 100, 110, 120, 130], crs=ccrs.PlateCarree())
#axc.gridlines()

plt.title('Ensemble mean PREC, 850hPa winds \n Forecast start: %s, Lead: T+%s h \n Valid at %s' % (date_label, (t * 24), valid_date_label))

bounds = np.arange(2,22,2)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    
# overplot wind as streamlines
cl = plt.gca().streamplot(X, Y, u850_ens_mean[t].data, v850_ens_mean[t].data, 
                          density=(2., 1.5), maxlength=1, norm=norm,
                          cmap='GnBu',
                          color=speed_ens_mean[t].data, linewidth=lw)

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

font = {'color':  'red', 'weight': 'bold', 'size': 12}
axc.text(91, 19, "COLD SURGE: {:.1f}%".format(cs_prob[t]*100.), fontdict=font, verticalalignment='top', bbox=props, alpha=0.7)

font = {'color':  'red', 'weight': 'bold', 'size': 12}
axc.text(91, 17.5, "CROSS-EQUATORIAL SURGE: {:.1f}%".format(ces_prob[t]*100.), fontdict=font, verticalalignment='top', bbox=props, alpha=0.7)
        
colorbar_axes = plt.gcf().add_axes([0.2, 0.12, 0.6, 0.025])
colorbar = plt.colorbar(cf, colorbar_axes, orientation='horizontal', label='Precip (mm day$^{-1}$)')

colorbar_axes = plt.gcf().add_axes([0.92, 0.25, 0.025, 0.5])
colorbar = plt.colorbar(cl.lines, colorbar_axes, orientation='vertical', label='Wind Speed (m s$^{-1}$)')

NameError: name 'X' is not defined

In [12]:
print("COLD SURGE: {:.1f}%".format(cs_prob[t]*100.))

NameError: name 'cs_prob' is not defined

In [188]:
for t in np.arange(ntimes):
    valid_date = date + datetime.timedelta(days=int(t))
    print(valid_date)

2021-01-01 12:00:00
2021-01-02 12:00:00
2021-01-03 12:00:00
2021-01-04 12:00:00
2021-01-05 12:00:00
2021-01-06 12:00:00
2021-01-07 12:00:00
2021-01-08 12:00:00


In [72]:
result = precip_cube.collapsed('realization', iris.analysis.PERCENTILE, percent=[10])
qplt.contourf(result[2])
plt.gca().coastlines()

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


In [74]:
result = speed_cube.collapsed('realization', iris.analysis.PERCENTILE, percent=[90])
qplt.contourf(result[2])
plt.gca().coastlines()

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


In [36]:
t = 1
precip_threshold = 50
speed_threshold = 10
precip_prob = precip_cube.collapsed('realization', iris.analysis.PROPORTION, 
                               function=lambda values: values > precip_threshold)

speed_prob = speed_cube.collapsed('realization', iris.analysis.PROPORTION, 
                               function=lambda values: values > speed_threshold)

fig = plt.figure(figsize=(10, 10), dpi=100)

try:
    # contour plot of precip
    clevels = np.arange(0.1, 1.1, 0.1)
    cf = iplt.contourf(precip_prob[t], levels=clevels, cmap='gray_r', alpha=0.65, extend='max')
    axc = plt.gca()
except:
    axc = plt.subplot(111, projection=ccrs.PlateCarree())
    
axc.coastlines('50m', alpha=0.5)
axc.set_ylim([-10, 20])
axc.set_xlim([90, 130])
axc.stock_img()
axc.set_yticks([-10, 0, 10, 20], crs=ccrs.PlateCarree())
axc.set_xticks([90, 100, 110, 120, 130], crs=ccrs.PlateCarree())

'''
axc = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
#axc.set_extent([90, 130, -10, 20], crs=ccrs.PlateCarree())
axc.coastlines('50m', alpha=0.5)
axc.set_ylim([-10, 20])
axc.set_xlim([90, 130])
axc.set_yticks([-10, 0, 10, 20], crs=ccrs.PlateCarree())
axc.set_xticks([90, 100, 110, 120, 130], crs=ccrs.PlateCarree())
axc.stock_img()
'''
#axc.gridlines()

clevels = np.arange(0.5, 1.01, 0.1)
cfc = iplt.contour(speed_prob[t], levels=clevels, linewidths=clevels*3, extend='max', alpha=0.7)
#plt.gca().coastlines()

title = 'Ensemble probability PREC, 850hPa winds \n Forecast start: %s, Lead: T+%s h \n Valid at %s' % (date_label, (t * 24), valid_date_label)
#title += '\n Precip. prob (p$\geq$ %s mm day$^{-1}$)' %precip_threshold
#title += '\n Wind speed. prob (p$\geq$ %s m s$^{-1}$)' %speed_threshold

plt.title(title)
colorbar_axes = plt.gcf().add_axes([0.2, 0.12, 0.6, 0.025])
colorbar = plt.colorbar(cf, colorbar_axes, orientation='horizontal', label='Precip. prob (p$\geq$ %s mm day$^{-1}$)' %precip_threshold)

colorbar_axes = plt.gcf().add_axes([0.91, 0.2, 0.025, 0.2])
cb2 = plt.colorbar(cfc, colorbar_axes, orientation='vertical', label='Wind speed. prob (p$\geq$ %s m s$^{-1}$)' %speed_threshold)
cb2.outline.set_edgecolor('white')
# set colorbar ticklabels
cb2.ax.tick_params(color="white")

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/

In [32]:
precip_prob = precip_cube.collapsed('realization', iris.analysis.PROPORTION, 
                               function=lambda values: values > 2000)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


In [36]:
iplt.contourf(precip_prob[3])

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


In [39]:
#mask = u850_ens_mean[0].data.copy()
dummy = speed_ens_mean.copy()
mask1 = (u850_ens_mean.data >= 0.) #& 
mask2 = (v850_ens_mean.data >= 0.) #& 
mask3 = (speed_ens_mean.data >= 5.)
#mask = (speed_ens_mean.data < 10.)
dummy.data = np.ma.array(speed_ens_mean.data, mask=mask1*mask2*mask3)
qplt.contourf(dummy[0])
plt.gca().coastlines()

In [85]:
for member in range(10):
    t = 1
    plt.quiver(X[:1,:1], Y[:1,:1], u850_cube[t,member, :1,:1].data, v850_cube[t,member, :1,:1].data, alpha=0.2)

In [83]:
#u850_cube

X Wind (m s-1),forecast_period,realization,latitude,longitude
Shape,8,36,214,178
Dimension coordinates,,,,
forecast_period,x,-,-,-
realization,-,x,-,-
latitude,-,-,x,-
longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_reference_time,x,x,-,-
time,x,x,-,-
Scalar coordinates,,,,


In [455]:
def enumerated_product(*args):
    yield from zip(itertools.product(*(range(len(x)) for x in args)), itertools.product(*args))
    

def cold_surge_probabilities(u850_cube, v850_cube, speed_cube):
    # Cold surge identification
    chang_box = [107, 115, 5, 10]        # CP Chang's 2nd domain

    # Hattori box for cross equatorial surges
    hattori_box = [105, 115, -5, 5]

    Chang_threshold = 9.0 # 10 # wind speed m/s
    Hattori_threshold = -2.0 # m/s meridional wind
    
    u850_ba = u850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
    u850_ba = u850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

    v850_ba = v850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
    v850_ba = v850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

    speed_ba = speed_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
    speed_ba = speed_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)
    
    # Hattori index
    v850_hattori = v850_cube.intersection(latitude=(hattori_box[2], hattori_box[3]), longitude=(hattori_box[0], hattori_box[1]))
    v850_hattori = v850_hattori.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

    # extract the forecast periods and members from the data
    # to create a metrics array
    forecast_periods = u850_cube.coord('forecast_period').points
    members = u850_cube.coord('realization').points
    
    # Check for cross-equatorial surges
    # CP index
    mask1 = u850_ba.data>0.
    mask2 = v850_ba.data>0.
    mask3 = speed_ba.data<=Chang_threshold
    mask4 = v850_ba.data>Hattori_threshold

    speed_ma = speed_ba.data.copy()
    speed_ma = np.ma.array(speed_ma, mask=mask1)
    speed_ma = np.ma.array(speed_ma, mask=mask2)
    cs_metric = np.ma.array(speed_ma, mask=mask3)

    #CES Hattori index
    ces_metric = np.ma.array(v850_ba.data, mask=mask4)
    ces_metric = np.ma.array(ces_metric, mask=cs_metric.mask)
    
    # return the probabilities as fraction
    return cs_metric.count(axis=1)/float(len(members)), ces_metric.count(axis=1)/float(len(members))

In [458]:
cs_prob, ces_prob = cold_surge_probabilities(u850_cube, v850_cube, speed_cube)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: User

In [468]:
plt.plot(cs_prob*100.)
plt.plot(ces_prob*100.)

In [382]:
u850_ba = u850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
u850_ba = u850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

v850_ba = v850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
v850_ba = v850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

speed_ba = speed_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
speed_ba = speed_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

# Hattori index
v850_hattori = v850_cube.intersection(latitude=(hattori_box[2], hattori_box[3]), longitude=(hattori_box[0], hattori_box[1]))
v850_hattori = v850_hattori.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: User

In [88]:
Chang_threshold = 9.0 # 10 # wind speed m/s
Hattori_threshold = -2.0 # m/s meridional wind

mask1 = u850_ens_mean.data>0.
mask2 = v850_ens_mean.data>0.
mask3 = speed_ens_mean.data<=Chang_threshold
mask4 = v850_ens_mean.data>Hattori_threshold

#np.ma.array(speed_ba.data, mask=)
speed_ma = speed_ens_mean.data.copy()

speed_ma = np.ma.array(speed_ma, mask=mask1)
speed_ma = np.ma.array(speed_ma, mask=mask2)

cs_metric = speed_ens_mean.copy()
cs_metric.data = np.ma.array(speed_ma, mask=mask3)

plt.subplot(111)
qplt.pcolormesh(cs_metric[0])
plt.gca().coastlines()

In [92]:
#CES
speed_ces = speed_ens_mean.data.copy()
speed_ces = np.ma.array(speed_ces, mask=mask1)
speed_ces = np.ma.array(speed_ces, mask=mask2)
#speed_ces = np.ma.array(speed_ces, mask=mask3)
speed_ces = np.ma.array(speed_ces, mask=mask4)

ces_metric = speed_ens_mean.copy()
ces_metric.data = np.ma.array(speed_ces, mask=mask4)
#ces_metric.data = np.ma.array(ces_metric.data, mask=cs_metric.data.mask)

plt.subplot(111)
qplt.pcolormesh(ces_metric[2])
plt.gca().coastlines()
#plt.colorbar()

In [51]:
speed_ma.shape

(8, 214, 178)

In [454]:
plt.plot(cs_metric.count(axis=1)/36.)
plt.plot(ces_metric.count(axis=1)/36.)

In [438]:
ces_metric

masked_array(
  data=[[13.235036849975586, 14.363262176513672, 14.666253089904785,
         13.021166801452637, 13.721320152282715, 14.505474090576172,
         13.244720458984375, 13.512650489807129, 14.00009536743164,
         13.12613582611084, 12.500503540039062, 12.252906799316406,
         12.868427276611328, 13.483177185058594, 13.812342643737793,
         14.335355758666992, 14.637784004211426, 13.426780700683594,
         13.101456642150879, 12.796355247497559, 11.484163284301758,
         14.578365325927734, 13.632922172546387, 14.634739875793457,
         13.361306190490723, 13.512377738952637, 13.58327865600586,
         14.964446067810059, 13.053930282592773, 12.60180377960205,
         13.359545707702637, 14.29128646850586, 13.702561378479004,
         14.877312660217285, 13.386419296264648, 13.491727828979492],
        [11.847515106201172, 11.118597030639648, 12.604354858398438,
         12.100317001342773, 12.882773399353027, 13.888492584228516,
         13.276514053344

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [355]:
plt.pcolormesh(ces_metric)
plt.colorbar()

In [296]:
qplt.plot(cs_prob)